In [2]:
# READ IN THE CSV

import pandas as pd
import numpy as np

df_sf = pd.read_csv('Smart_Farm_dataset.csv')

In [1]:
# DO ANY PRE OBSERVATION HERE

df_sf

NameError: name 'df_sf' is not defined

In [3]:
# ONE HOT REPRESENT (PREVIOUSLY) STRING ENCODED LOC AND GEN COLUMNS

df_sf = pd.concat([df_sf, pd.get_dummies(df_sf['Loc'])], axis=1)
df_sf = pd.concat([df_sf, pd.get_dummies(df_sf['Gen'])], axis=1)

# DROP ANY COLUMNS HERE
COLUMNS_DROPPED = ['Date', 'Loc', 'Gen', 'PlantID']
df_sf = df_sf.drop(COLUMNS_DROPPED, axis=1)

In [ ]:
df_sf.loc[90:100]

In [4]:
# THIS MODULE WILL SPLIT UP EACH 100 POINTS IN THE DATAFRAME INTO SEPARATE CYCLES SO THAT WE CAN DO TIME SHIFTING
# ON THE APPROPRIATE CYCLES
cycles = []
for i in range(0, 4):
    start_index = i*100
    end_index = start_index + 99
    df_cycle = df_sf.loc[start_index:end_index]
    cycles.append(df_cycle)

In [54]:
# data.reset_index().drop(['index'], axis=1).to_csv('reindexed_output_shifted_cycles.csv')
new_data = data.reset_index().drop(['index'], axis=1)
X_train = new_data.loc[0:84]

In [69]:
[i for i in range(0,5)] + [5]

[0, 1, 2, 3, 4, 5]

In [94]:
X_train = new_data.iloc[[i for i in range(0,85)]
                        + [i for i in range(90,175)]
                        + [i for i in range(180, 265)]
                        + [i for i in range(270,355)]]
y_train = X_train.loc[:, 'var2(t)']
X_train = X_train.loc[:,~data.columns.str.contains('\(t\)')]
# X = data.drop([data.columns.str.contains('\(t\)')], axis=1)
X_test = new_data.iloc[[i for i in range(85,90)]
                        + [i for i in range(175,180)]
                        + [i for i in range(265,270)]
                        + [i for i in range(355, 360)]]
y_test = X_test.loc[:, 'var2(t)']
X_test = X_test.loc[:,~data.columns.str.contains('\(t\)')]

In [95]:
X_test

,var1(t-10),var2(t-10),var3(t-10),var4(t-10),var5(t-10),var6(t-10),var7(t-10),var8(t-10),var9(t-10),var10(t-10),...,var1(t-1),var2(t-1),var3(t-1),var4(t-1),var5(t-1),var6(t-1),var7(t-1),var8(t-1),var9(t-1),var10(t-1)
85,86.0,6.670000e-05,95.0,23.47,61.17,0.003958,1.0,0.0,1.0,0.0,...,95.0,1.270000e-07,76.0,25.06,67.93,0.001042,1.0,0.0,1.0,0.0
86,87.0,3.330000e-05,90.0,24.05,58.65,0.000208,1.0,0.0,1.0,0.0,...,96.0,6.330000e-08,83.0,25.45,67.36,0.000417,1.0,0.0,1.0,0.0
87,88.0,1.660000e-05,83.0,23.69,62.14,0.000000,1.0,0.0,1.0,0.0,...,97.0,3.160000e-08,82.0,25.60,64.60,0.000417,1.0,0.0,1.0,0.0
88,89.0,8.270000e-06,92.0,23.58,60.59,0.000000,1.0,0.0,1.0,0.0,...,98.0,1.570000e-08,89.0,25.56,69.67,0.000208,1.0,0.0,1.0,0.0
89,90.0,4.120000e-06,96.0,24.34,65.34,0.000000,1.0,0.0,1.0,0.0,...,99.0,7.850000e-09,89.0,29.70,81.26,0.000000,1.0,0.0,1.0,0.0
175,86.0,5.140000e-07,95.0,23.47,61.17,0.003958,1.0,0.0,0.0,1.0,...,95.0,1.300000e-10,76.0,25.06,67.93,0.001042,1.0,0.0,0.0,1.0
176,87.0,2.050000e-07,90.0,24.05,58.65,0.000208,1.0,0.0,0.0,1.0,...,96.0,5.180000e-11,83.0,25.45,67.36,0.000417,1.0,0.0,0.0,1.0
177,88.0,8.160000e-08,83.0,23.69,62.14,0.000000,1.0,0.0,0.0,1.0,...,97.0,2.060000e-11,82.0,25.60,64.60,0.000417,1.0,0.0,0.0,1.0
178,89.0,3.250000e-08,92.0,23.58,60.59,0.000000,1.0,0.0,0.0,1.0,...,98.0,8.220000e-12,89.0,25.56,69.67,0.000208,1.0,0.0,0.0,1.0
179,90.0,1.300000e-08,96.0,24.34,65.34,0.000000,1.0,0.0,0.0,1.0,...,99.0,3.280000e-12,89.0,29.70,81.26,0.000000,1.0,0.0,0.0,1.0


In [ ]:
# FORMER TRAIN AND TEST SPLIT APPORACH

X_train = pd.DataFrame()
y_train = np.array([])
X_test = pd.DataFrame()
y_test = np.array([])

true_index_counter = -1
for i in range (0, int(len(df_sf)/100)):
    for k in range (1, 101):
        true_index_counter += 1
        if k <= 95:
            X_train = X_train.append(df_sf.loc[true_index_counter].drop(['GrowthRate']))
            y_train = np.append(y_train, df_sf.loc[true_index_counter, 'GrowthRate'])
        else:
            X_test = X_test.append(df_sf.loc[true_index_counter].drop(['GrowthRate']))
            y_test = np.append(y_test, df_sf.loc[true_index_counter, 'GrowthRate'])

In [5]:
from pandas import DataFrame
from pandas import concat

def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    """
    Frame a time series as a supervised learning dataset.
    Arguments:
        data: Sequence of observations as a list or NumPy array.
        n_in: Number of lag observations as input (X).
        n_out: Number of observations as output (y).
        dropnan: Boolean whether or not to drop rows with NaN values.
    Returns:
        Pandas DataFrame of series framed for supervised learning.
    """
    n_vars = 1 if type(data) is list else data.shape[1]
    df = DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg




# print(data)
# raw = DataFrame()
# raw['ob1'] = [x for x in range(10)]
# raw['ob2'] = [x for x in range(50, 60)]
# values = raw.values
# data = series_to_supervised(values, 1, 19)
# print(data)
# data

# X_train = data.drop(['var2(t)'], axis=1)
# y_train = data.loc[:, 'var2(t)']
# print([data if val == True else None for ind, val in enumerate(data.columns.str.contains('\(t\)'))])
# for ind, val in enumerate(data.columns.str.contains('\(t\)')):
#     if val == True:
#         data.drop([])
        

    

In [6]:
data = pd.DataFrame()
for df in cycles:
    values = df.values
    values_supervised = series_to_supervised(values, 10, 1)
    data = pd.concat([data, values_supervised], axis=0)
    
    


In [ ]:
values = df_sf.values
data = series_to_supervised(values,10,1)

In [ ]:
data.to_csv('output_shifted_cycles.csv')

In [34]:
#TRAIN AND TEST SPLIT

X = data.loc[:,~data.columns.str.contains('\(t\)')]
# X = data.drop([data.columns.str.contains('\(t\)')], axis=1)
y = data.loc[:, 'var2(t)']

from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.9, test_size = 0.1)


In [ ]:
X_train.columns.str.contains('\(t\)')

In [12]:
# ERROR COMPUTATION FUNCTIONS

from sklearn.metrics import mean_squared_error
def evaluate_errors(prediction, actual):
    print("RMSE Error: ", np.sqrt(mean_squared_error(prediction, actual)))
    avg_error_vector = np.absolute(((prediction - actual) / actual) * 100)
    print("Average Error details:\n", np.mean(avg_error_vector))
    return avg_error_vector

In [9]:
# FEATURE IMPORTANCE FUNCTIONS

def get_feature_importances(regr):
    feature_importances = regr.feature_importances_
    feature_importances = pd.Series(feature_importances)
    feature_importance_df = pd.DataFrame({'feature': X_train.columns,'feature_importance': feature_importances})
    feature_importance_df = feature_importance_df.sort_values(by=['feature_importance'])
    for index, row in feature_importance_df.iterrows():
        print(row['feature'], 'has importance: ', row['feature_importance'])

In [96]:
# MODELS RF

from sklearn.ensemble import RandomForestRegressor
regr = RandomForestRegressor(n_estimators=20, max_depth=13, random_state=0, verbose=1, n_jobs=-1)
regr.fit(X_train, y_train)
preds = regr.predict(X_test)

preds = pd.DataFrame(preds)
y_test = pd.DataFrame(y_test)
# evaluate_errors(preds, y_test)
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
print('MAE', mean_absolute_error(preds, y_test))
print('RMSE', np.sqrt(mean_squared_error(preds, y_test)))
get_feature_importances(regr)

[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:    0.1s finished
[Parallel(n_jobs=4)]: Done  20 out of  20 | elapsed:    0.0s finished


MAE 0.05035181879900358
RMSE 0.08141561469059781
var7(t-10) has importance:  7.779705197766933e-08
var10(t-6) has importance:  8.624568322857439e-08
var9(t-5) has importance:  1.0127658964550529e-07
var10(t-9) has importance:  3.242182905383565e-07
var10(t-1) has importance:  3.2780355002755236e-07
var9(t-3) has importance:  5.347805579600149e-07
var7(t-5) has importance:  5.454867764674887e-07
var9(t-1) has importance:  6.229154230608587e-07
var8(t-2) has importance:  7.298841791177522e-07
var8(t-8) has importance:  8.940478972473184e-07
var7(t-7) has importance:  1.1324108581726458e-06
var10(t-8) has importance:  1.1471578838950498e-06
var7(t-8) has importance:  2.199870662537143e-06
var7(t-4) has importance:  2.6109013648232764e-06
var8(t-1) has importance:  2.811595398879157e-06
var9(t-2) has importance:  2.962061789803085e-06
var10(t-5) has importance:  3.103755743187172e-06
var9(t-10) has importance:  3.145949392675189e-06
var7(t-2) has importance:  3.3624142372985677e-06
var7(t-

In [ ]:
print(pd.concat([preds, y_test.astype(float).reset_index()], axis=1 ))

In [ ]:
data.loc[19,data.columns.str.contains('var2')]

In [ ]:
# MODELS LIGHTGBM

from lightgbm import LGBMRegressor

clf = LGBMRegressor(n_estimators=1000, learning_rate=0.01)
clf.fit(X_train, np.log1p(y_train))
preds = np.expm1(clf.predict(X_test))

preds = pd.DataFrame(preds)
y_test = pd.DataFrame(y_test)

evaluate_errors(preds, y_test)

In [ ]:
# preds = model.predict(X_test)
preds = model.predict(X_train)

In [ ]:
print(preds)
errs = evaluate_errors(preds, y_train)
print(errs)

In [ ]:
from sklearn.neural_network import MLPRegressor

clf = MLPRegressor(solver='adam', activation='tanh')
clf.fit(X_train, y_train)
preds = clf.predict(X_test)
# evaluate_errors(preds, y_test)
# print(preds)
# print(y_train)

from sklearn.metrics import mean_squared_error
print(mean_squared_error(preds, y_test))
# get_feature_importances(clf)

In [ ]:
from sklearn.neural_network import MLPRegressor

clf = MLPRegressor(solver='lbfgs', activation='tanh')
clf.fit(X_train, y_train)
preds = clf.predict(X_test)
evaluate_errors(preds, y_test)
print(preds)
print(y_test)

In [ ]:
#THIS IS AN EXAMPLE OF A TIME SERIES PROBLEM WITH KERAS

import numpy
import matplotlib.pyplot as plt
from pandas import read_csv
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Flatten
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
# # convert an array of values into a dataset matrix
# def create_dataset(dataset, look_back=1):
# 	dataX, dataY = [], []
# 	for i in range(len(dataset)-look_back-1):
# 		a = dataset[i:(i+look_back), 0]
# 		dataX.append(a)
# 		dataY.append(dataset[i + look_back, 0])
# 	return numpy.array(dataX), numpy.array(dataY)
# # fix random seed for reproducibility
# numpy.random.seed(7)
# # load the dataset
# # dataframe = read_csv('Syngenta/Syngenta_2017/Experiment_dataset.csv', engine='python', skipfooter=3)
# dataset = dataframe.values
# dataset = dataset.astype('float32')
# # normalize the dataset
# scaler = MinMaxScaler(feature_range=(0, 1))
# dataset = scaler.fit_transform(dataset)
# # split into train and test sets
# train_size = int(len(dataset) * 0.67)
# test_size = len(dataset) - train_size
# train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]
# # reshape into X=t and Y=t+1
look_back = 12
# trainX, trainY = create_dataset(train, look_back)
# testX, testY = create_dataset(test, look_back)
# # reshape input to be [samples, time steps, features]
# trainX = numpy.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
# testX = numpy.reshape(testX, (testX.shape[0], 1, testX.shape[1]))
# create and fit the LSTM network
model = Sequential()
model.add(LSTM(4, input_shape=(380, look_back)))
# model.add(Flatten())
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(X_train, y_train, epochs=100, batch_size=1, verbose=2)
# make predictions
trainPredict = model.predict(X_train)
testPredict = model.predict(X_test)
# invert predictions
trainPredict = scaler.inverse_transform(trainPredict)
trainY = scaler.inverse_transform([trainY])
testPredict = scaler.inverse_transform(testPredict)
testY = scaler.inverse_transform([testY])
# calculate root mean squared error
trainScore = math.sqrt(mean_squared_error(trainY[0], trainPredict[:,0]))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = math.sqrt(mean_squared_error(testY[0], testPredict[:,0]))
print('Test Score: %.2f RMSE' % (testScore))
# shift train predictions for plotting
trainPredictPlot = numpy.empty_like(dataset)
trainPredictPlot[:, :] = numpy.nan
trainPredictPlot[look_back:len(trainPredict)+look_back, :] = trainPredict
# shift test predictions for plotting
testPredictPlot = numpy.empty_like(dataset)
testPredictPlot[:, :] = numpy.nan
testPredictPlot[len(trainPredict)+(look_back*2)+1:len(dataset)-1, :] = testPredict
# plot baseline and predictions
plt.plot(scaler.inverse_transform(dataset))
plt.plot(trainPredictPlot)
plt.plot(testPredictPlot)
plt.show()

In [ ]:
#THIS IS AN EXAMPLE OF A TIME SERIES PROBLEM WITH KERAS

# Stacked LSTM for international airline passengers problem with memory
import numpy
import matplotlib.pyplot as plt
from pandas import read_csv
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
# convert an array of values into a dataset matrix
def create_dataset(dataset, look_back=1):
	dataX, dataY = [], []
	for i in range(len(dataset)-look_back-1):
		a = dataset[i:(i+look_back), 0]
		dataX.append(a)
		dataY.append(dataset[i + look_back, 0])
	return numpy.array(dataX), numpy.array(dataY)
# fix random seed for reproducibility
numpy.random.seed(7)
# load the dataset
dataframe = read_csv('international-airline-passengers.csv', usecols=[1], engine='python', skipfooter=3)
dataset = dataframe.values
dataset = dataset.astype('float32')
# normalize the dataset
scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(dataset)
# split into train and test sets
train_size = int(len(dataset) * 0.67)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]
# reshape into X=t and Y=t+1
look_back = 3
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)
# reshape input to be [samples, time steps, features]
trainX = numpy.reshape(trainX, (trainX.shape[0], trainX.shape[1], 1))
testX = numpy.reshape(testX, (testX.shape[0], testX.shape[1], 1))
# create and fit the LSTM network
batch_size = 1
model = Sequential()
model.add(LSTM(4, batch_input_shape=(batch_size, look_back, 1), stateful=True, return_sequences=True))
model.add(LSTM(4, batch_input_shape=(batch_size, look_back, 1), stateful=True))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
for i in range(100):
	model.fit(trainX, trainY, epochs=1, batch_size=batch_size, verbose=2, shuffle=False)
	model.reset_states()
# make predictions
trainPredict = model.predict(trainX, batch_size=batch_size)
model.reset_states()
testPredict = model.predict(testX, batch_size=batch_size)
# invert predictions
trainPredict = scaler.inverse_transform(trainPredict)
trainY = scaler.inverse_transform([trainY])
testPredict = scaler.inverse_transform(testPredict)
testY = scaler.inverse_transform([testY])
# calculate root mean squared error
trainScore = math.sqrt(mean_squared_error(trainY[0], trainPredict[:,0]))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = math.sqrt(mean_squared_error(testY[0], testPredict[:,0]))
print('Test Score: %.2f RMSE' % (testScore))
# shift train predictions for plotting
trainPredictPlot = numpy.empty_like(dataset)
trainPredictPlot[:, :] = numpy.nan
trainPredictPlot[look_back:len(trainPredict)+look_back, :] = trainPredict
# shift test predictions for plotting
testPredictPlot = numpy.empty_like(dataset)
testPredictPlot[:, :] = numpy.nan
testPredictPlot[len(trainPredict)+(look_back*2)+1:len(dataset)-1, :] = testPredict
# plot baseline and predictions
plt.plot(scaler.inverse_transform(dataset))
plt.plot(trainPredictPlot)
plt.plot(testPredictPlot)
plt.show()

In [ ]:
#THIS IS AN EXAMPLE OF A TIME SERIES PROBLEM WITH KERAS

from keras.models import Sequential
from keras.layers import Dense
from sklearn import datasets
import numpy
# fix random seed for reproducibility
numpy.random.seed(7)
# load pima price dataset
dataset = datasets.load_boston()
# split into input (X) and output (Y) variables
# X = dataset[:,0:8]
# Y = dataset[:,8]
# create model
model = Sequential()
model.add(Dense(12, input_dim=109, activation='relu'))
model.add(Dense(8, activation='relu'))
# model.add(Dense(1, activation='sigmoid'))

model.add(Dense(1, activation='linear'))
model.compile(loss='mse', optimizer='adam')
# model.fit(X, y, epochs=1000, verbose=0)

# Compile model
# model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# Fit the model
model.fit(X_train, y_train, epochs=150, batch_size=10)
# evaluate the model
scores = model.evaluate(X_train, y_train)
# print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))